In [2]:
REPO_LINKS = {
    "abinit": {
        "url": "https://github.com/abinit/abinit",
        "lang": "fortran"
    },
    "mdanalysis": {
        "url": "https://github.com/MDAnalysis/mdanalysis",
        "lang": "python"
    },
    "libmesh": {
        "url": "https://github.com/libMesh/libmesh",
        "lang": "C"
    },
    "lammps": {
        "url": "https://github.com/lammps/lammps",
        "lang": "C"
    },
    "amber": {
        "url": "https://github.com/Amber-MD",
        "lang": "C"
    },
    "hoomd": {
        "url": "https://github.com/joaander/hoomd-blue/tree/master/hoomd",
        "lang": "C"
    },
    "pcmsolver": {
        "url": "https://github.com/PCMSolver/pcmsolver",
        "lang": "C"
    },
    "rmg-py": {
        "url": "https://github.com/ReactionMechanismGenerator/RMG-Py",
        "lang": "python"
    },
    "xenon": {
        "url": "https://github.com/NLeSC/Xenon",
        "lang": "Java"
    }
}


In [3]:
import os, sys, shlex
import pandas as pd
import numpy as np
from pdb import set_trace
import warnings
from datetime import datetime
import dateutil
from pathlib import Path

warnings.filterwarnings("ignore")
main_path = os.getcwd()
print(main_path)
p_df = {}

/home/huyqt7/Projects/PhD/data-collection/labeled_commits


In [51]:
import subprocess as sp

for p in REPO_LINKS.keys():
    commits_dir = main_path + "/preprocessed_commits/" + p
    files = [filename for filename in os.listdir(commits_dir) if filename.endswith(".csv")]
    files = sorted(files)
    dfs = []
    for i in range(len(files)):
        df = pd.read_csv(commits_dir + "/" + files[i])
        if i == 0:
            dfs = [df['hash'].iloc[0], df['hash'].iloc[-1]]
        else:
            dfs.append(df['hash'].iloc[-1])
    print(p)
    project_path = "/home/huyqt7/Projects/PhD/eager/data/projects/%s/" % p
    os.chdir(project_path)
    re_commits = []
    for index in range(len(dfs)):
        commit_cmd = "git reset --hard {}".format(dfs[index])
        sp.call(commit_cmd, shell=True)
        path = "/home/huyqt7/Projects/PhD/data-collection/labeled_commits/release_level/%s/%s_%s.udb" % (p, p, index) 
        if REPO_LINKS[p]["lang"] == "fortran":
            cmd = "und create -db {} -languages Fortran add {}".format(str(path), str(project_path))
            sp.call(cmd, shell=True)
            cmd = "und -db {} settings -FileTypes .F90=Fortran .F77=Fortran .F03=Fortran .F95=Fortran .F=Fortran".format(str(path))
            sp.call(cmd, shell=True)
            cmd = "und -db {} analyze -all".format(str(path))
        elif REPO_LINKS[p]["lang"] == "python":
            cmd = "und create -languages python add {} analyze {}".format(
                str(project_path), str(path))
        elif REPO_LINKS[p]["lang"] == "C":
            cmd = "und create -languages C++ add {} analyze {}".format(
                str(project_path), str(path))
        elif REPO_LINKS[p]["lang"] == "Java":
            cmd = "und create -languages Java add {} analyze {}".format(
                str(project_path), str(path))
        sp.call(cmd, shell=True)
        #print(commit_cmd, cmd, dfs[index], p)

abinit
mdanalysis
libmesh
lammps
amber
hoomd
pcmsolver
rmg-py
xenon


In [52]:
import math 
import understand as und
for p in ['abinit']:
    commits_dir = main_path + "/release_level/" + p
    files = [filename for filename in os.listdir(commits_dir) if filename.endswith(".udb")]
    no_files = len(files)
    for i in range(no_files):
        u_fname = "%s/release_level/%s/%s_%s.udb" % (main_path, p, p, i)
        m_fname = "%s/release_level/%s/%s_%s_file_metrics.csv" % (main_path, p, p, i)
        #print(u_fname)
        db = und.open(u_fname)
        metrics_dataframe = pd.DataFrame()
        for f in db.ents("File"):
            metrics = f.metric(f.metrics())
            metrics["File"] = f.name()
            metrics["Bugs"] = 0
            metrics_dataframe = metrics_dataframe.append(pd.Series(metrics), ignore_index=True)
        columns_order = ['File'] + [a for a in metrics_dataframe.columns if a not in ['File', 'Bugs']] + ['Bugs']
        metrics_dataframe = metrics_dataframe.reindex(columns=(columns_order))
        metrics_dataframe = metrics_dataframe.drop_duplicates(subset='File', keep="last").reset_index()
        metrics_dataframe = metrics_dataframe.dropna()
        metrics_dataframe.to_csv(m_fname, index=False)
        db.close()

In [59]:
def _os_cmd(cmd, verbose=False):
        """
        Run a command on the shell
        Parameters
        ----------
        cmd: str
            A command to run.
        """
        cmd = shlex.split(cmd)
        with sp.Popen(cmd, stdout=sp.PIPE, stderr=sp.DEVNULL) as p:
            out, err = p.communicate()

        if verbose:
            print(out)
            print(err)
        return out, err

def _files_changed_in_git_diff(hash_1, hash_2, p):
        """
        Get a list of all the files changed between two hashes
        Parameters
        ----------
        hash_1 : str
            Commit hash 1.
        hash_2 : bool
            Commit hash 2.
        Returns
        -------
        List[str]:
            A list of all files changed. For simplicity we only include *.py
            *.F90, *.c, *.cpp, *.java.
        """
        project_path = "/home/huyqt7/Projects/PhD/eager/data/projects/%s/" % p
        os.chdir(project_path)
        sp.call("git reset --hard master", shell=True)
        out, __ = _os_cmd("git diff {}..{} --name-only".format(hash_1, hash_2))
        files_changed = []
        for f in out.splitlines():
            if f and "__init__.py" not in str(f):
                type_f = str(f)[2:-1].split(".")
                #print(str(f), type_f)
                if type_f:
                    if type_f[-1] in set(["c", "cc", "cxx", "C", "cpp", "CPP", "h", "H", 
                                   "f90", "f", "f77", "f03", "f95", "for", "ftn",
                                   "F90", "F", "F77", "F03", "F95", "For", "Ftn", 
                                   "py", "java"]):
                        file_name = str(f)
                        #print(file_name)
                        if type_f[-1][0] == "F":
                            file_name[-4] == "f"
                            files_changed.append(Path(file_name).name[:-1])
                        else:
                            files_changed.append(Path(file_name).name[:-1])
                

        # A work around for FORTRAN file extensions.
        #if REPO_LINKS[p]["lang"] == "fortran":
        #    files_changed = list(map(lambda x: x[:-4]+".f90", files_changed))
        return files_changed

In [85]:
import math 
import subprocess as sp
for p in ['mdanalysis', 'libmesh', 'lammps']:
    print(p, main_path)
    commits_dir = main_path + "/preprocessed_commits/" + p
    files = [filename for filename in os.listdir(commits_dir) if filename.endswith(".csv")]
    no_versions = len(files)
    files_changed = [] 
    for index in range(no_versions, 0, -1):
        c_fname = "%s/preprocessed_commits/%s/%s_%s.csv" % (main_path, p, p, index)
        df_commits = pd.read_csv(c_fname, index_col=None)
        fixed_bugs_indices = df_commits[df_commits['fastread_buggy'] == 1].index.values.tolist()
        fixed_bugs_indices = [idx for idx in fixed_bugs_indices if idx > 1]
        previous_bugs_indices = [idx-1 for idx in fixed_bugs_indices]
        fixed_bugs = df_commits.loc[fixed_bugs_indices]['hash'].values
        previous_bugs = df_commits.loc[previous_bugs_indices]['hash'].values
        m_fname = "%s/release_level/%s/%s_%s_file_metrics.csv" % (main_path, p, p, index-1)
        df_metrics = pd.read_csv(m_fname)
        df_metrics['fastread_bugs'] = [0]*df_metrics.shape[0]
        print(index, len(fixed_bugs_indices), end=", ")
        #df_metrics['Bugs'] = [0] * df_metrics.shape[0]
        for hash_id, prev_id in zip(fixed_bugs, previous_bugs):
            if not files_changed:
                files_changed = _files_changed_in_git_diff(prev_id, hash_id, p)
            else:
                files_changed.extend(_files_changed_in_git_diff(prev_id, hash_id, p))
            #print(files_changed)
        for changed_item in files_changed:
            changed_i_exist = list(df_metrics[df_metrics['File'] == changed_item].index)
            if changed_i_exist:
                df_metrics.loc[changed_i_exist, 'fastread_bugs'] += 1
        df_metrics.to_csv(m_fname[:-4] + "_1.csv", index=False)
    print()

mdanalysis /home/huyqt7/Projects/PhD/data-collection/labeled_commits
8 190, 7 74, 6 68, 5 70, 4 72, 3 100, 2 97, 1 95, 
libmesh /home/huyqt7/Projects/PhD/data-collection/labeled_commits
7 165, 6 129, 5 159, 4 199, 3 235, 2 236, 1 236, 
lammps /home/huyqt7/Projects/PhD/data-collection/labeled_commits
8 132, 7 98, 6 114, 5 94, 4 88, 3 44, 2 22, 1 5, 


In [84]:
for p in ['mdanalysis', 'libmesh', 'lammps']:
    print(p, main_path)
    commits_dir = main_path + "/preprocessed_commits/" + p
    commits_fs = [filename for filename in os.listdir(commits_dir) if filename.endswith(".csv")]
    commits_fs = sorted(commits_fs)
    fr_df = pd.read_csv(main_path + "/fastread/" + p + "_fast_labeled.csv")
    fr_df['fr_labels'] = pd.Series(np.where(fr_df.code == 'yes', 1, 0), fr_df.index)
    for f in commits_fs:
        df = pd.read_csv(commits_dir + "/" + f, index_col=None)
        human_labels = [0] * df.shape[0]
        for index, row in df.iterrows():
            h = str(row['hash'])
            if len(h) > 7:
                h = h[:7]
            val = fr_df[fr_df['Document Title'].str.startswith(h)]
            if not val.empty:
                for i, row in val.iterrows():
                    if str(row['Document Title'])[-2:] != ".0":            
                        human_labels[index] = val['fr_labels'].values.tolist()[-1]
        print(len(human_labels), df['fastread_buggy'].shape[0]) 
        df['fastread_buggy'] = human_labels
        df.to_csv(commits_dir + "/" + f, index=False)

mdanalysis /home/huyqt7/Projects/PhD/data-collection/labeled_commits
352 352
339 339
392 392
355 355
426 426
429 429
403 403
648 648
libmesh /home/huyqt7/Projects/PhD/data-collection/labeled_commits
1125 1125
1326 1326
1339 1339
947 947
1062 1062
983 983
1897 1897
lammps /home/huyqt7/Projects/PhD/data-collection/labeled_commits
807 807
883 883
914 914
1180 1180
855 855
754 754
837 837
1284 1284


In [75]:
a = [0, 1, 2, 3, 4]
a[-2:]

[3, 4]